## 1. Imports & Setup

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from scipy.stats import pearsonr

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Base raw data directory (notebooks → ../data/raw)
DATA_DIR = os.path.join("..", "data", "raw")

# CSV paths
TRAIN_CSV = os.path.join(DATA_DIR, "csvs", "train.csv")
TEST_CSV  = os.path.join(DATA_DIR, "csvs", "test.csv")

# Audio directories
TRAIN_AUDIO_DIR = os.path.join(DATA_DIR, "audios", "train")
TEST_AUDIO_DIR  = os.path.join(DATA_DIR, "audios", "test")

# Feature file (created in Day 2)
FEATURES_CSV = os.path.join("..", "data", "processed", "train_audio_features.csv")

## 2. Load Features & Labels

In [8]:
# Load extracted features
X = pd.read_csv(FEATURES_CSV, index_col=0)

# Load training labels
train_df = pd.read_csv(TRAIN_CSV)
y = train_df["label"]

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)

Feature matrix shape: (276, 34)
Target vector shape: (409,)
